In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

In [8]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

In [9]:
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

In [10]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)

In [11]:
train_dataset[0]

(tensor([[[-1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
           -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
           -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
           -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000],
          [-1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
           -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
           -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
           -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000],
          [-1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
           -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
           -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
           -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000],
          [-1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
           -1.0000, -1.0000, -1.000

In [12]:
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        # input layer
        self.fc1 = nn.Linear(28*28, 128)
        # output layer
        self.fc2 = nn.Linear(128, 64)

    # Feed Forward
    def forward(self, x):
      # Turn image into 1D
      x = x.view(-1, 28*28)
      # Hidden Layer: ReLu(Activation Function)
      x = F.relu(self.fc1(x))
      # Output Layer
      x = F.relu(self.fc2(x))
      # Using SoftMax for Output Layer
      return F.log_softmax(x, dim=1)

In [13]:
model = SimpleNN()

# Loss Function and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

In [14]:
# Model Training
def train(model, device, train_loader, optimizer, epoch):
  model.train()
  for batch_idx, (data, target) in enumerate(train_loader):
    data, target = data.to(device), target.to(device)
    optimizer.zero_grad()
    output = model(data)
    loss = criterion(output, target)
    loss.backward()
    optimizer.step()
    if batch_idx % 100 == 0:
      print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} '
            f'({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')


In [15]:
# Model Testing
def test(model, device, test_loader):
  model.eval()
  test_loss = 0
  correct = 0
  with torch.no_grad():
    for data, target in test_loader:
      data, target = data.to(device), target.to(device)
      output = model(data)
      test_loss += criterion(output, target).item()  # 손실 합산
      pred = output.argmax(dim=1, keepdim=True)  # 가장 높은 확률을 가진 클래스 예측
      correct += pred.eq(target.view_as(pred)).sum().item()  # 맞춘 개수 합산

  test_loss /= len(test_loader.dataset)
  test_accuracy = 100. * correct / len(test_loader.dataset)
  print(f'Test set: Average loss: {test_loss:.4f}, Accuracy: {test_accuracy:.2f}%')
  return test_loss, test_accuracy

In [19]:
## Training & Testing Execution
# Setting
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

epochs = 1
train_losses, test_losses, test_accuracies = [], [], []

# Excute!
for epoch in range(1, epochs + 1):
  train(model, device, train_loader, optimizer, epoch)
  test_loss, test_accuracy = test(model, device, test_loader)
  test_losses.append(test_loss)
  test_accuracies.append(test_accuracy)

Train Epoch: 1 [0/60000 (0%)]	Loss: 0.158166
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.163301
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.155440
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.108277
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.114343
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.210668
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.079261
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.301955
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.103289
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.212476
Test set: Average loss: 0.0001, Accuracy: 96.25%


In [17]:
# Summary of the trained model
from torchsummary import summary
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                  [-1, 128]         100,480
            Linear-2                   [-1, 64]           8,256
Total params: 108,736
Trainable params: 108,736
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.41
Estimated Total Size (MB): 0.42
----------------------------------------------------------------
